In [ ]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

In [ ]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

In [ ]:
file_name = "food_entities__20180321.p"   
df_retrived = f.retrive_file(file_name)
df_retrived.reset_index(inplace=True)

In [4]:
print df_retrived.shape
print df_retrived.columns.tolist()
df_retrived.head(1)

(105218, 13)
['index', 'assigned_name', 'name_variants', 'menu_name', 'tag', 'price', 'desc', 'vendors', 'identification_index', 'loc', 'timestamp', 'price_average', 'review_loc']


,index,assigned_name,name_variants,menu_name,tag,price,desc,vendors,identification_index,loc,timestamp,price_average,review_loc
0,food_item_0,,"[, afternoon set for, ala carte, ala carte set...","[(Set 1, 646), (Set 2, 633), (Set 4, 599), (Se...","[ Fragrance Mala Hotpot, CHINESE NEW YEAR Sp...","[1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, ...","[, \t1 Caesar Salad,\n\t1 Mesclun Salad,\n\t2 ...","[delivery_0, delivery_10, delivery_1001, deliv...","(food_0, food_522, food_948, food_961, food_11...","[deliveroo/food/1000, deliveroo/food/1001, del...","[2017-06-05T07:25:24.501860, 2017-06-07 01:35:...",26.952902,[]


** Food-restaurant edges **

Food offered by the restaurant based on delivery menus, or mentioned in reviews of the restaurant

In [5]:
%%time
# food restaurant_delivery_boolean
edge_type = "food_vendor"
ref_restaurant_food = df_retrived.set_index("index").to_dict()["vendors"]
res = dict()
index = 0
for food in df_retrived["index"].tolist():
    for r in ref_restaurant_food[food]:
            res.update({"_".join([edge_type,str(index)]):{
                            "index":"_".join([edge_type,str(index)]),
                        "from_id": food, 
                        "weight":1, 
                        "to_id":r, 
                        "type": edge_type}})
            index+=1
print "number of %s relations found: %d"%(edge_type,index)


number of food_vendor relations found: 418083
CPU times: user 1.07 s, sys: 36 ms, total: 1.1 s
Wall time: 1.11 s


In [6]:
# print sample
from itertools import islice
dict(islice(res.iteritems(), 0, 2))

{'food_vendor_362100': {'from_id': 'food_item_90191',
  'index': 'food_vendor_362100',
  'to_id': 'delivery_3799',
  'type': 'food_vendor',
  'weight': 1},
 'food_vendor_362101': {'from_id': 'food_item_90191',
  'index': 'food_vendor_362101',
  'to_id': 'delivery_6138',
  'type': 'food_vendor',
  'weight': 1}}

In [7]:
# write file
file_name = 'food_vendor.json'
import json
with open(file_name, 'w') as outfile:
    json.dump(json.dumps(res), outfile)
print "saved: %s"%file_name

saved: food_vendor.json


** Resraurant-restaurant edges **

Number of common food offered by restaurants

In [8]:
%%time
# restaurant_delivery, restaurant_delivery, freq
edge_type = "vendor_vendor"
ref_restaurant_food = df_retrived.set_index("index").to_dict()["vendors"]
# frquency of vendor-vendor pair
rr_pair = f.flatten([f.combi(l) for l in ref_restaurant_food.values()])
rr_freq = f.count_freq([tuple(sorted(l)) for l in rr_pair])
# writing file
res2 = dict()
index = 0
for freq in rr_freq:
    res2.update({"_".join([edge_type,str(index)]):{
        "index":"_".join([edge_type,str(index)]),
        "from_id": freq[0][1], 
        "weight":freq[1], 
        "to_id":freq[0][0], 
        "type": edge_type}})
    index+=1
    res2.update({"_".join([edge_type,str(index)]):{
        "index":"_".join([edge_type,str(index)]),
        "from_id": freq[0][0], 
        "weight":freq[1], 
        "to_id":freq[0][1], 
        "type": edge_type}})
    index+=1
print "number of %s relations found: %d"%(edge_type,index)

number of vendor_vendor relations found: 17286436
CPU times: user 1min 26s, sys: 6.07 s, total: 1min 32s
Wall time: 1min 31s


In [9]:
# print sample
from itertools import islice
dict(islice(res2.iteritems(), 0, 2))

{'vendor_vendor_6968562': {'from_id': 'delivery_84',
  'index': 'vendor_vendor_6968562',
  'to_id': 'delivery_3943',
  'type': 'vendor_vendor',
  'weight': 2},
 'vendor_vendor_6968563': {'from_id': 'delivery_3943',
  'index': 'vendor_vendor_6968563',
  'to_id': 'delivery_84',
  'type': 'vendor_vendor',
  'weight': 2}}

In [10]:
# write file
file_name = 'vendor_vendor.json'

import json
with open(file_name, 'w') as outfile:
    json.dump(json.dumps(res2), outfile)
print "saved: %s"%file_name

saved: vendor_vendor.json


** module for writing file to elastic search **